In [ ]:
import pandas as pd
import json

In [ ]:
data = pd.DataFrame(columns=['date', 'indice'])
with open('../data/st_test.json', 'r') as file:
        d = json.loads(file.read())

        data['date'] = d['Indices']['NDVI']['Serie Processada']['Data']
        data['indice'] = d['Indices']['NDVI']['Serie Processada']['Indice']

In [ ]:
data